In [33]:
import cohere
import json
import io
import warnings

import pandas as pd
from IPython.display import display
from PIL import Image

from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation

import sys
if '..' not in sys.path:
    sys.path.append('..')
    
import os
import random


from app.story_generator.segmentation import StoryDivider

In [110]:
def get_random_story():
    stories_path = os.path.join(os.path.abspath(''), os.pardir,
                                "data", "stories", "fairy_tales.json")
    with open(stories_path, "rb") as f:
        stories = json.load(f)
        example_story = random.choice(stories)
    return example_story


def get_segmented_story(n_pages=None, sentences_per_page=3):
    story = get_random_story()
    seg = StoryDivider(story["title"], story["text"])
    segmented = seg.divide_story_into_segments(n_pages=n_pages, sentences_per_page=sentences_per_page)
    # print(segmented)
    return seg.story_id, segmented

# APIs

In [111]:
with open('../cohere_api_key.txt', 'r') as f:
    cohere_api_key = f.read()
co = cohere.Client(cohere_api_key)

with open('../stability_api_key.txt', 'r') as f:
    stability_api_key = f.read()
stability_api = client.StabilityInference(
    key=stability_api_key, 
    verbose=True,
)

del cohere_api_key
del stability_api_key

# EDA

In [112]:
with open('../data/stories/fairy_tales.json') as f:
    data = json.load(f)
    # text = data['text']
    # print(type(text), len(text))
data = data[0]
title = data['title']
text = data['text']
text

'Once upon a time, a brother and sister named Hansel and Gretel lived in a hut in the woods with their father who was a poor woodcutter and their mother. Their parents were very poor and had barely enough food to eat One day, their parents sent them off into the woods in search of greener pastures. Their mother cried as she sent them off but they could not take care of them any longer. Hansel and Gretel took a few pebbles and some bread crumbs with them. That night the two children had nowhere to sleep. They wandered in the forest for days, looking for food and a place to stay. The children slept under a tree that night and kept each other warm. The next morning when the sun rose, Hansel turned to his little sister. “Gretel,” he said, “we cannot stay here.  We must go deeper into the woods!  Surely we will find more to eat than what we get at home and we can bring some food for mother and father” Gretel was worried. “But what if we get lost?” she asked “We won’t!” said Hansel.  “If we 

In [114]:
story_title_1, random_story_1 = get_segmented_story(sentences_per_page=5)
random_story_2 = random_story_1

while random_story_1[0] == random_story_2[0]:
    story_title_2, random_story_2 = get_segmented_story(sentences_per_page=5)

# Generating stories

In [71]:
def generate(prompt, model="xlarge", 
             num_generations=5, temperature=0.7, 
             max_tokens=2000, stop_sequences=['<end>']):
             
  prediction = co.generate(
    model=model,
    prompt=prompt,
    return_likelihoods = 'GENERATION',
    stop_sequences=stop_sequences,
    max_tokens=max_tokens,
    temperature=temperature,
    num_generations=num_generations)
  
  # Get list of generations
  gens = []
  likelihoods = []
  for gen in prediction.generations:
      gens.append(gen.text)
      
      sum_likelihood = 0
      for t in gen.token_likelihoods:
          sum_likelihood += t.likelihood
      # Get sum of likelihoods
      likelihoods.append(sum_likelihood)

  pd.options.display.max_colwidth = 200
  # Create a dataframe for the generated sentences and their likelihood scores
  df = pd.DataFrame({'generation': gens, 'likelihood': likelihoods})
  # Drop duplicates
  df = df.drop_duplicates(subset=['generation'])
  # Sort by highest sum likelihood
  df = df.sort_values('likelihood', ascending=False, ignore_index=True)
  
  return df

In [65]:
def generate_image(image_prompt):
  # the object returned is a python generator
  answers = stability_api.generate(
      prompt=image_prompt
  )

  # iterating over the generator produces the api response
  for resp in answers:
      for artifact in resp.artifacts:
          if artifact.finish_reason == generation.FILTER:
              warnings.warn(
                  "Your request activated the API's safety filters and could not be processed."
                  "Please modify the prompt and try again.")
          if artifact.type == generation.ARTIFACT_IMAGE:
              img = Image.open(io.BytesIO(artifact.binary))
              display(img)


In [138]:
def add_newline_at_the_end(text):
    if text[-1] != '\n':
        text += '\n'
    return text

def generate_prompt_for_story_start(header: str,  examples: list, titles: list = None, story_title: str = None,
                                    pre_example_string: str = 'Answer: ', stop_token: str ='<end>', max_tokens: int = 1000):
    prompt = add_newline_at_the_end(header)
    
    for i, ex in enumerate(examples):
        if titles is not None:
            prompt += 'Title: ' + add_newline_at_the_end(titles[i])
        prompt += pre_example_string
        prompt += add_newline_at_the_end(ex)
        prompt += add_newline_at_the_end(stop_token)
        
    if titles is not None:
        prompt += 'Title: '
        if story_title is not None:
            prompt += add_newline_at_the_end(story_title)
            prompt += pre_example_string
    else:
        prompt += pre_example_string
    
    
    estimated_tokens_number = len(prompt.split(' ')) + len(ex.split(' '))
    estimated_tokens_number *= 2 # let's assume word is on averate 2 tokens
    assert estimated_tokens_number < max_tokens, f'Estimated number of tokens was {estimated_tokens_number} which is more than specified max number of tokens ({max_tokens})' 
    
    return prompt

In [141]:
header = 'Exercise: Generate the beginning of the story for children based on a title given.'
examples = [random_story_1[0], random_story_2[0]]
titles = [story_title_1, story_title_2]
story_title = 'The Dragon and prince penguin' # should be None if there is 
MAX_TOKENS = 1000

In [134]:
prompt = generate_prompt_for_story_start(header, examples, titles=titles, story_title=story_title)

In [135]:
print(prompt)

Exercise: Generate the beginning of the story for children based on a title given.
Title: The Three Little Pigs
Answer: Once upon a time, there was a farmer with three little pigs. He did not have enough food to take care of his pigs so he sent them away to take care of themselves. The first little pig was walking on the road. Suddenly he saw a man with some straws. He could  build a house with the straws, he said to himself and asked the man to give him his straws
<end>
Title: Hansel and Gretel
Answer: Once upon a time, a brother and sister named Hansel and Gretel lived in a hut in the woods with their father who was a poor woodcutter and their mother. Their parents were very poor and had barely enough food to eat One day, their parents sent them off into the woods in search of greener pastures. Their mother cried as she sent them off but they could not take care of them any longer. Hansel and Gretel took a few pebbles and some bread crumbs with them. That night the two children had n

In [136]:
result = generate(
    prompt,
    max_tokens=1000
)

In [143]:
for i, row in result.iterrows():
    print('-'*50)
    print('likelihood:', row['likelihood'])
    text = row['generation']
    if story_title is not None:
        text = add_newline_at_the_end(story_title) + text
    print(text)

--------------------------------------------------
likelihood: -53.08446738279999
The Dragon and prince penguin
 Once upon a time, a dragon was sleeping in his cave. A prince penguin came and woke him up. He was hungry. The dragon took him to a restaurant and ordered some food for him. The prince penguin thanked the dragon for his kindness.
<end>
--------------------------------------------------
likelihood: -76.0008235151
The Dragon and prince penguin
 Once upon a time, there was a dragon who was bored and lonely. He wanted a friend to play with him. One day, he met a penguin and wanted to be friends with him. The dragon took the penguin to his cave. They were very happy to play together. However, the penguin was afraid of the fire coming out of the dragon's mouth
<end>
--------------------------------------------------
likelihood: -108.261479235
The Dragon and prince penguin
 Once upon a time, there was a prince and a dragon. The dragon was very mean. The prince's father had told him

In [132]:
print(result.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   generation  5 non-null      object 
 1   likelihood  5 non-null      float64
dtypes: float64(1), object(1)
memory usage: 208.0+ bytes
None


In [93]:
result.iloc[0]['generation']

'Once upon a time, there was a little girl who lived with her mother very close to the woods. The little girl was called Little Red Riding Hood because of the red hat her grandmother gifted her. She liked this hat very much and wore it everywhere. Her mother told her not to talk to strangers and she did as told. Whenever she was sent on an errand, she would be careful not to talk to strangers\n<end>'

In [68]:
print(result['generation'][0])

 and her lost dog that is a fascinating read. This short story
is one of those stories that keeps you hooked from the very beginning. It
captures your attention and your imagination and makes you want to read more.

I liked the story, the story of a young girl
who is looking for her dog. The girl goes to find her dog and meets different
people on the way. The story is told from the point of view of the girl and
through the voice of the dog. The story is well written and I liked the way the
author wrote it.

This story is about a girl named Molly who
has a dog named Lola. Molly's dog is lost and she goes to find her dog. She
meets different people on the way and they help her. The story is written in a
very simple and interesting way. The story is about the dog and the girl. The
author has written the story in a very simple and interesting way. The story is
very well written and it is very interesting. The story is written in a very
simple and interesting way. The story is about the dog

In [69]:
test_string = """
Once upon a time, there was a farmer with three little pigs. He did not have enough food to take care of his pigs so he sent them away to take care of themselves. The first little pig was walking on the road. Suddenly he saw a man with some straws. He could  build a house with the straws, he said to himself and asked the man to give him his straws. The man was kind so he gave the first little pig his straws. The pig used the straws to build a straw house and danced around. Suddenly, a big bad wo
"""

In [70]:
len(test_string)

502